# Jeopardy! Project

## Project Goals

You will work to write several functions that investigate a dataset of Jeopardy!
questions and answers. Filter the dataset for topics that you’re interested in,
compute the average difficulty of those questions, and train to become
the next Jeopardy champion!

## Prerequisites

1. In order to complete this project, you should have completed the Pandas lessons in the [Analyze Data with Python Skill Path](https://www.codecademy.com/learn/paths/analyze-data-with-python). You can also find those lessons in the [Data Analysis with Pandas](https://www.codecademy.com/learn/data-processing-pandas) course or the [Data Scientist Career Path](https://www.codecademy.com/learn/paths/data-science/). 
    
    Finally, the [Practical Data Cleaning](https://www.codecademy.com/learn/practical-data-cleaning) course may also be helpful.

## Project Requirements
2. We’ve provided a csv file containing data about the game show Jeopardy! in a file named jeopardy.csv. Load the data into a DataFrame and investigate its contents. Try to print out specific columns.

    Note that in order to make this project as “real-world” as possible, we haven’t modified the data at all — we’re giving it to you exactly how we found it. As a result, this data isn’t as “clean” as the datasets you normally find on Codecademy. More specifically, there’s something odd about the column names. After you figure out the problem with the column names, you may want to rename them to make your life easier the rest of the project.

    In order to disply the full contents of a column, we’ve added this line of code to the top of your file:
    ```python
    pd.set_option('display.max_colwidth', -1)
    ```

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

jeopardy = pd.read_csv('jeopardy.csv')

In [2]:
print(jeopardy.columns)

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')


In [3]:
renamed_columns = {}
for column in jeopardy.columns:
    renamed_columns[column] = column.lstrip()
jeopardy.rename(renamed_columns, axis='columns', inplace=True)
# print(jeopardy.columns)

In [4]:
for column in jeopardy.columns:
    if jeopardy[column].dtype == 'object':
        jeopardy[column] = jeopardy[column].str.strip()

In [5]:
jeopardy.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,"No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves",Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,"The city of Yuma in this state has a record average of 4,055 hours of sunshine each year",Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", this company served its billionth burger",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Constitution of Mass., second President of the United States",John Adams


3. Write a function that filters the dataset for questions that contains all of the words in a list of words. For example, when the list `["King", "England"]` was passed to our function, the function returned a DataFrame of 152 rows. Every row had the strings `"King"` and `"England"` somewhere in its `" Question"`.

    Note that in this example, we found 152 rows by filtering the entire dataset. You can download the entire dataset at the start or end of this project. The dataset used on Codecademy is only a fraction of the dataset so you won’t find as many rows.

    Test your function by printing out the column containing the question of each row of the dataset.

In [6]:
# First version.
# def question_filter(dataframe, word_list):
#     """
#     Return a new pandas DataFrame where its 'Question' column contains all
#     of the strings in word_list.
#     """
#     new_df = dataframe[dataframe.Question.apply(lambda question: all([(word.lower() in question.lower()) for word in word_list]))]
    
#     return new_df

4. Test your original function with a few different sets of words to try to find some ways your function breaks. Edit your function so it is more robust.

    For example, think about capitalization. We probably want to find questions that contain the word `"King"` or `"king"`.

    You may also want to check to make sure you don’t find rows that contain substrings of your given words. For example, our function found a question that didn’t contain the word `"king"`, however it did contain the word `"viking"` — it found the `"king"` inside `"viking"`. Note that this also comes with some drawbacks — you would no longer find questions that contained words like `"England's"`.

In [7]:
# Second version.
def question_filter(dataframe, word_list):
    """
    Return a new pandas DataFrame where its 'Question' column contains all
    of the strings in word_list.
    """
    if isinstance(word_list, str):
        new_df = dataframe[dataframe.Question.apply(lambda question: word_list.lower() in question.lower().split())]
    else:
        new_df = dataframe[dataframe.Question.apply(lambda question: all([(word.lower() in question.lower().split()) for word in word_list]))]
    
    return new_df

5. We may want to eventually compute aggregate statistics, like `.mean()` on the `" Value"` column. But right now, the values in that column are strings. Convert the `" Value"` column to floats. If you’d like to, you can create a new column with the float values.

    Now that you can filter the dataset of question, use your new column that contains the float values of each question to find the “difficulty” of certain topics. For example, what is the average value of questions that contain the word `"King"`?

    Make sure to use the dataset that contains the float values as the dataset you use in your filtering function.

In [8]:
def string_to_float(string):
    numeric_characters = []
    for character in string:
        if character.isdecimal() or character in '.':
            numeric_characters.append(character)
    new_string = ''.join(numeric_characters)
    
    try:
        return float(new_string)
    except:
        return float('nan')

jeopardy['Value_float'] = jeopardy.Value.apply(string_to_float)

Example:

In [9]:
question_filter(jeopardy, ['King']).Value_float.mean()

818.8878000979912

6. Write a function that returns the count of the unique answers to all of the questions in a dataset. For example, after filtering the entire dataset to only questions containing the word `"King"`, we could then find all of the unique answers to those questions. The answer “Henry VIII” appeared 3 times and was the most common answer.

In [10]:
def unique_answers(dataframe):
    """
    Return a count of all the unique answers of the dataframe.
    """
    unique_answer_count = dataframe.Answer.value_counts()
    
    return unique_answer_count

In [11]:
unique_answers(question_filter(jeopardy, "King"))

Henry VIII                                      41
Sweden                                          24
Solomon                                         23
Norway                                          22
Richard III                                     21
                                                ..
James I (also known as James VI of Scotland)     1
Meneleus                                         1
the Sierra Nevadas                               1
King Solomon                                     1
the Equator                                      1
Name: Answer, Length: 1165, dtype: int64

7. Explore from here! This is an incredibly rich dataset, and there are so many interesting things to discover. There are a few columns that we haven’t even started looking at yet. Here are some ideas on ways to continue working with this data:

    * Investigate the ways in which questions change over time by filtering by the date. How many questions from the 90s use the word "Computer" compared to questions from the 2000s?
    * Is there a connection between the round and the category? Are you more likely to find certain categories, like "Literature" in Single Jeopardy or Double Jeopardy?
    * Build a system to quiz yourself. Grab random questions, and use the input function to get a response from the user. Check to see if that response was right or wrong. Note that you can’t do this on the Codecademy platform — to do this, download the data, and write and run the code on your own computer!

In [12]:
science_questions = jeopardy[jeopardy.Category.str.contains('SCIENCE', case=False)]

In [13]:
sherlock_holmes_questions = jeopardy[jeopardy.Category.str.contains('HOLMES', case=False)]

In [14]:
print(len(science_questions))
print(len(sherlock_holmes_questions))

2230
29


In [15]:
# star_wars_categories = ['LUKE', 'HAND, SOLO', 'CHEWY', 'BUBBA FETE', '"R"2 "D"2']
# star_wars_questions = jeopardy[jeopardy.Category.apply(lambda x: x in star_wars_categories)]

In [16]:
unique_answers(question_filter(jeopardy, "Rock"))

Elvis Presley          16
limestone               9
Gibraltar               7
Sisyphus                7
Bob Dylan               5
                       ..
bullets                 1
Goldie's oldies         1
Central High School     1
Sex Pistols             1
Courtney Love           1
Name: Answer, Length: 578, dtype: int64

In [17]:
jeopardy[jeopardy.Answer == "limestone"]

,Show Number,Air Date,Round,Category,Value,Question,Answer,Value_float
3444,5347,2007-12-04,Double Jeopardy!,GEOLOGY,$1200,"This rock can be formed by the accumulation of shells or coral, but not from citrus fruit",limestone,1200.0
4999,308,1985-11-13,Jeopardy!,ROCKS & MINERALS,$500,"Fizzing when acid is applied, this mineral is the base of the Portland cement industry",limestone,500.0
42868,5180,2007-03-02,Jeopardy!,"THE FIRST ""STONE""",$1000,"(<a href=""http://www.j-archive.com/media/2007-03-02_J_05.jpg"" target=""_blank"">Jeff Probst reports from Guatemala.</a>) Mayan pyramids were characteristically constructed with <a href=""http://www.j-archive.com/media/2007-03-02_J_05a.jpg"" target=""_blank"">this</a> stone, named for the calcium oxide it produces when burned",limestone,1000.0
56312,4145,2002-09-13,Double Jeopardy!,GEOLOGY TEST,$1600,"Calcium bicarbonate from this, the most abundant soluble rock, is used by sea creatures in forming shells",limestone,1600.0
79138,5225,2007-05-04,Jeopardy!,REEF MADNESS,$800,"The skeletons of coral polyps are made of calcium carbonate, so coral reefs are mainly this sedimentary rock",limestone,800.0
86867,4594,2004-07-22,Jeopardy!,SEDIMENTARY ROCK,$1000,"(<a href=""http://www.j-archive.com/media/2004-07-22_J_29.wmv"">Cheryl of the Clue Crew dissolves a piece of rock in a chemistry experiment.</a>) With diluted hydrochloric acid, you can test for this rock; the acid reacts with the calcite to form CO<sub>2</sub>",limestone,1000.0
101498,5858,2010-02-17,Double Jeopardy!,ROCK,$1600,"In its purest form, this rock used in the cement industry contains only calcite",limestone,1600.0
110745,5572,2008-11-25,Double Jeopardy!,CAVES,$1200,"Nevada's formation-rich Lehman Caves are made of low-grade marble, better known as this",limestone,1200.0
125533,3845,2001-04-27,Double Jeopardy!,I AM A ROCK,$1000,"The Rock of Gibraltar is a huge block of this rock made of calcite, a mineral form of calcium carbonate",limestone,1000.0
151655,1907,1992-12-15,Double Jeopardy!,THE EARTH,$600,"Dolomitization is the process by which this rock, including its fossils, turns into dolomite",limestone,600.0


In [18]:
rock_questions = jeopardy[
    jeopardy.Category.str.contains('ROCK|STONE|GEOLOGY|REEF', regex=True)
    | jeopardy.Category.apply(lambda category: category in ["SCULPTURE", "CAVES"])
]

In [19]:
rock_questions.Category.unique()

array(['ALBUMS THAT ROCK', 'ROCK WITH YOU', 'MEDICAL MILESTONES',
       'RHYMES WITH STONEHENGE', 'ROCK MUSIC', '"ROCK"Y', 'ROCK-Y',
       "IT'S ONLY ROCK & ROLL", 'GEOLOGY', 'CORAL REEF LIFE', 'SCULPTURE',
       'ROCKS & MINERALS', "ROLLING STONE'S 100 GREATEST GUITARISTS",
       'THE KIDS LOVE THAT ROCK & ROLL', 'ROCK & ROLL FRONTMEN',
       'ROCK FORMATIONS', 'ROCK OF STAGES', 'THE ROLLING STONES',
       'BIRTHSTONES', 'ROCK SONGS', 'THE ROCKETS', 'BRIT ROCK',
       'ALTERNATE LITERARY TOMBSTONES', 'ROCK ME',
       "ROCK'S FRONTMEN & WOMEN", 'COLORFUL ROCK', 'ROCK ‘N ROLL',
       'ALL OF OUR "STONE"', 'ON THE COVER OF ROLLING STONE',
       'POP & ROCK', 'INDIE ROCK', 'TELEVISION MILESTONES', "'80s ROCK",
       'ROCK & ROLL JEOPARDY!', 'GEMSTONES', 'CLEVELAND ROCKS!',
       'BROTHERLY ROCK', 'MICROBIOLOGY MILESTONES', 'NAME THE ROCKERS',
       'SCHOOL OF ROCK MOVIES', 'THEY WERE STONED',
       'ROCK BAND COUNTRY OF ORIGIN', 'ROCK & ROLL HISTORY', "'60s ROCK",
       'LE

In [20]:
from random import randrange


def string_unformatted(string):
    stripped_string = string.strip()
    unformatted_string = ''.join(stripped_string.lower().split())
    return unformatted_string


def get_question_and_answer(dataframe, value=0.0, category=''):
    """"""
    question_df = dataframe
    if value:
        question_df = question_df[question_df.Value_float == value]
    if category:
        question_df = question_df[question_df.Category == category]
        
    if len(question_df) > 1:
        number_of_questions = len(question_df)
        row = question_df.iloc[randrange(number_of_questions)]
    else:
        row = question_df.iloc[0]
    
    return row


def ask_question_get_answer(row):
    category = row.Category
    value = row.Value
    question = row.Question
    answer = row.Answer
    answer_unformatted = string_unformatted(answer)
    guess = string_unformatted(input(''.join([f"{category} for {value}:", "\n\n", question, "\n"])))
    if guess == answer_unformatted:
        print("That's correct!")
    else:
        print(f"Incorrect. The correct answer was {answer}.")


def jeopardy_quiz(dataframe):
    """"""
    question, answer = get_question_and_answer(dataframe, value, category)
    pass

Example play:

In [21]:
ask_question_get_answer(get_question_and_answer(jeopardy))

THEME PARK THRILLERS for $500:

With "Mantis" making an even dozen, Cedar Point in this state has the most roller coasters of any park
 Ohio


That's correct!
